In [ ]:
import codecs
import os
import sys
import threading
import serial
from serial.tools.list_ports import comports

1. Gimbal control

In [ ]:
from a2gmeasurements import GimbalRS2
import time, timeit
import numpy as np

myGimbal = GimbalRS2()
myGimbal.start_thread_gimbal()

In [ ]:
from a2gmeasurements import GimbalRS2
import time, timeit
import numpy as np

myGimbal = GimbalRS2()
myGimbal.start_thread_gimbal()

assumed_time_for_execute_time = 0.0015 # s
gimbal_speed = (1800/3.47) # DEG/s
condition_continue = True
tol = gimbal_speed * assumed_time_for_execute_time
cnt = 0
myGimbal.request_current_position()
time.sleep(0.0015)
last_yaw = myGimbal.yaw 
last_roll = myGimbal.roll
print('Outside: ',last_yaw, last_roll)
myGimbal.setPosControl(yaw=1250, roll=0, pitch=0)
time.sleep(1)
while(condition_continue):
    print('Execution time: ', timeit.default_timer() - start)
    myGimbal.request_current_position()
    time.sleep(0.0015)
    print('Inside: ', myGimbal.yaw, myGimbal.roll)
    
    if np.abs(last_yaw - myGimbal.yaw) < tol and np.abs(last_roll - myGimbal.roll) < tol:
        condition_continue = False
    else:
        last_yaw = myGimbal.yaw
        last_roll = myGimbal.roll
        start = timeit.default_timer()
    cnt = cnt+1
    print(cnt)
    


In [ ]:
#myGimbal.request_current_position()
myGimbal.setPosControl(yaw=0, roll=0, pitch=0)

In [ ]:
myGimbal.stop_thread_gimbal()
time.sleep(0.05)
myGimbal.actual_bus.shutdown()

2. Anritsu Control

In [ ]:
from a2gmeasurements import myAnritsuSpectrumAnalyzer

mySA = myAnritsuSpectrumAnalyzer()
mySA.spectrum_analyzer_connect()
rx_power =  mySA.retrieve_max_pow()
print(rx_power)
mySA.spectrum_analyzer_close()

3. Septentrio Control

In [ ]:
from a2gmeasurements import GpsSignaling
mySeptentrioGPS = GpsSignaling(DBG_LVL_2=True, DBG_LVL_0=True)

In [ ]:
mySeptentrioGPS.serial_connect()
mySeptentrioGPS.start_thread_gps()
mySeptentrioGPS.start_gps_data_retrieval(msg_type='NMEA', nmea_type='GGA', interval='sec1') # HDF instead of GGA for heading info
#mySeptentrioGPS.start_gps_data_retrieval(msg_type='SBF')

In [ ]:
mySeptentrioGPS.stop_gps_data_retrieval(msg_type='NMEA')
mySeptentrioGPS.stop_thread_gps()

In [ ]:
print(len(mySeptentrioGPS.SBF_frame_buffer), len(mySeptentrioGPS.NMEA_buffer))
if len(mySeptentrioGPS.SBF_frame_buffer) > 0:
    print(mySeptentrioGPS.SBF_frame_buffer[-1])
if len(mySeptentrioGPS.NMEA_buffer) > 0:
    print(mySeptentrioGPS.NMEA_buffer[-1])

Precision of distance when using planar coordinates ('precision' is given by google estimate of distance)

In [ ]:
import pyproj as proj
from a2gmeasurements import HelperA2GMeasurements
import numpy as np

pt_hammentie_50 = [60.186977663598164, 24.961182204780805]
pt_hakaniemi_mkt = [60.17907971164074, 24.951228689442726]

h = HelperA2GMeasurements()

lat_pln_hamm, lon_pln_hamm = h.convert_DDMMS_to_planar(24.961182204780805, 60.186977663598164)
lat_pln_hak, lon_pln_hak = h.convert_DDMMS_to_planar(24.951228689442726, 60.17907971164074)

# Compare this distance with Google MAPS: CHECKED ---> within 2m error
d_hamm_hak = np.linalg.norm(np.array([lon_pln_hamm, lat_pln_hamm]) - np.array([lon_pln_hak, lat_pln_hak]))

Example of a real GPS NMEA output

In [ ]:
real_gps_data = {'Timestamp': '110815.00', 
                 'Latitude': '6011.1011606', 
                 'Latitude Direction': 'N', 
                 'Longitude': '02448.9144475', 
                 'Longitude Direction': 'E', 
                 'GPS Quality Indicator': '2', 
                 'Number of Satellites in use': '28', 
                 'Horizontal Dilution of Precision': '0.6', 
                 'Antenna Alt above sea level (mean)': '0.9230', 
                 'Units of altitude (meters)': 'M', 
                 'Geoidal Separation': '19.5883', 
                 'Units of Geoidal Separation (meters)': 'M', 
                 'Age of Differential GPS Data (secs)': '1.8', 
                 'Differential Reference Station ID': '0123'}

Server connection

In [ ]:
from socket import socket
import time
from a2gmeasurements import HelperA2GMeasurements

host = 'localhost'
myHelper = HelperA2GMeasurements('GROUND', host)
myHelper.HelperStartA2GCom()


In [ ]:
myHelper.HelperA2GStopCom()
print(myHelper.SOCKET_BUFFER, len(myHelper.SOCKET_BUFFER))

5. Signal generator

In [ ]:
import pyvisa

rm = pyvisa.ResourceManager()
print(rm.list_resources())


In [ ]:
inst = rm.open_resource('GPIB0::19::INSTR')
print(inst.query("*IDN?"))


In [ ]:
inst.write('F0 28.95 GH\n')
inst.write('L0 20 DM\n')

In [ ]:
# Turn On RF output
inst.write('RF1\n')

In [ ]:
# Turn Off RF output
inst.write('RF0\n')

In [1]:
from a2gmeasurements import myAnritsuSpectrumAnalyzer

dir = 'C://Users//jvjulian//OneDrive - Teknologian Tutkimuskeskus VTT//Documents//Measurement 1//20221012133438//'
file = 'CONT20221012133504968.rsm'

anritsu = myAnritsuSpectrumAnalyzer()
anritsu.parse_xml_file(dir + file)
anritsu.XML_file

{'Timestamp': 1665570904.0,
 'Data': [-61.53,
  -58.73,
  -58.37,
  -63.25,
  -60.2,
  -59.52,
  -59.43,
  -60.59,
  -58.83,
  -60.21,
  -62.16,
  -58.56,
  -55.29,
  -57.69,
  -57.28,
  -58.02,
  -60.27,
  -58.31,
  -57.97,
  -59.44,
  -58.33,
  -57.78,
  -57.94,
  -56.14,
  -55.15,
  -60.67,
  -57.59,
  -56.17,
  -55.98,
  -58.77,
  -59.77,
  -57.71,
  -57.77,
  -57.53,
  -59.65,
  -60.84,
  -59.67,
  -59.48,
  -61.94,
  -61.11,
  -57.97,
  -60.23,
  -60.16,
  -58.59,
  -58.18,
  -57.02,
  -58.05,
  -58.31,
  -59.03,
  -57.68,
  -57.05,
  -55.63,
  -58.04,
  -58.62,
  -54.49,
  -60.62,
  -61.57,
  -56.26,
  -57.58,
  -58.91,
  -59.48,
  -57.37,
  -61.32,
  -58.93,
  -57.05,
  -56.99,
  -56.6,
  -60.04,
  -57.54,
  -60.48,
  -56.38,
  -57.9,
  -58.59,
  -57.93,
  -57.63,
  -56.67,
  -55.75,
  -56.47,
  -61.39,
  -55.54,
  -55.34,
  -55.74,
  -57.67,
  -57.4,
  -55.77,
  -59.54,
  -57.66,
  -56.73,
  -58.12,
  -58.64,
  -55.19,
  -58.48,
  -57.35,
  -59.26,
  -56.67,
  -60.32,
  -56.08